In [1]:
import pandas as pd

# Load the WQI data, specifying that the header is in the third row (index 2)
wqi_df = pd.read_excel('wqi-data.xlsx', header=2)

# Load the nearest ponds distance data
nearest_ponds_df = pd.read_excel('nearest_ponds_distance.xlsx')

# Display the first few rows of each dataframe to verify they are loaded correctly
print("WQI Data Head:")
print(wqi_df.head())
print("\nNearest Ponds Data Head:")
print(nearest_ponds_df.head())

# Display the column names to check for any leading/trailing spaces
print("\nWQI Data Columns:")
print(wqi_df.columns)
print("\nNearest Ponds Data Columns:")
print(nearest_ponds_df.columns)

WQI Data Head:
  Date of Data Collection Measurement Type  Pond ID          Group  \
0              2024-10-31          Evening   2_78_1  Focus group 3   
1              2024-10-31          Evening  2_110_1  Focus group 4   
2              2024-10-31          Evening  2_108_1  Focus group 4   
3              2024-10-31          Morning   2_78_1  Focus group 3   
4              2024-10-31          Morning  2_110_1  Focus group 4   

  Time (sample or data collection)  Pond Type  \
0                         17:18:00  Treatment   
1                         17:03:00  Treatment   
2                         16:36:00  Treatment   
3                         07:32:00  Treatment   
4                         07:17:00  Treatment   

  FWI Staff Member Who Recorded Measurement  \
0                                    Chandu   
1                                    Chandu   
2                                    Chandu   
3                                    Chandu   
4                                 

In [2]:
# Strip any leading/trailing whitespace from column names
wqi_df.columns = wqi_df.columns.str.strip()
nearest_ponds_df.columns = nearest_ponds_df.columns.str.strip()

# Convert 'Date of Data Collection' to datetime objects
# The errors='coerce' will turn any unparseable dates into NaT (Not a Time)
wqi_df['Date of Data Collection'] = pd.to_datetime(wqi_df['Date of Data Collection'], errors='coerce')

# Drop rows where the date could not be parsed
wqi_df = wqi_df.dropna(subset=['Date of Data Collection'])

# Extract month from the 'Date of Data Collection'
wqi_df['Month'] = wqi_df['Date of Data Collection'].dt.month

# Display the first few rows with the new 'Month' column
print("WQI Data with Month Column:")
print(wqi_df.head())

# Also, let's process the 'kmz_file' column to get a numeric pond ID for merging
nearest_ponds_df['Pond ID'] = nearest_ponds_df['kmz_file'].str.replace('.kmz', '', regex=False).astype(int)
print("\nNearest Ponds Data with Pond ID:")
print(nearest_ponds_df.head())

WQI Data with Month Column:
  Date of Data Collection Measurement Type  Pond ID          Group  \
0              2024-10-31          Evening   2_78_1  Focus group 3   
1              2024-10-31          Evening  2_110_1  Focus group 4   
2              2024-10-31          Evening  2_108_1  Focus group 4   
3              2024-10-31          Morning   2_78_1  Focus group 3   
4              2024-10-31          Morning  2_110_1  Focus group 4   

  Time (sample or data collection)  Pond Type  \
0                         17:18:00  Treatment   
1                         17:03:00  Treatment   
2                         16:36:00  Treatment   
3                         07:32:00  Treatment   
4                         07:17:00  Treatment   

  FWI Staff Member Who Recorded Measurement  \
0                                    Chandu   
1                                    Chandu   
2                                    Chandu   
3                                    Chandu   
4                    

In [7]:
# Let's inspect the data types of the key columns to confirm the issue
print("Data types for wqi_df['Pond ID']:", wqi_df['Pond ID'].dtype)
print("Data types for nearest_ponds_df['Pond ID']:", nearest_ponds_df['Pond ID'].dtype)

# Convert 'Pond ID' in wqi_df to a numeric type.
# errors='coerce' will turn any non-numeric values into NaN (Not a Number)
wqi_df['Pond ID'] = pd.to_numeric(wqi_df['Pond ID'], errors='coerce')

# It's good practice to remove rows where 'Pond ID' could not be converted, as they are not useful
wqi_df.dropna(subset=['Pond ID'], inplace=True)

# Now, convert the 'Pond ID' column to integer type
wqi_df['Pond ID'] = wqi_df['Pond ID'].astype(int)

# Verify the change
print("\n--- After Correction ---")
print("New data type for wqi_df['Pond ID']:", wqi_df['Pond ID'].dtype)

Data types for wqi_df['Pond ID']: object
Data types for nearest_ponds_df['Pond ID']: int64

--- After Correction ---
New data type for wqi_df['Pond ID']: int64


In [8]:
# Merge the nearest_ponds_df with itself to get the WQI data for the main pond
merged_df = pd.merge(nearest_ponds_df, wqi_df, on='Pond ID', how='left')

# Rename columns for clarity before merging with neighbor data
merged_df = merged_df.rename(columns={
    'Dissolved Oxygen (mg/L)': 'Pond_Dissolved_Oxygen',
    'pH': 'Pond_pH',
    'Ammonia (mg/L)': 'Pond_Ammonia',
    'Date of Data Collection': 'Pond_Date'
})

# Merge with WQI data for the first nearest pond
merged_df = pd.merge(merged_df, wqi_df.add_suffix('_neighbor1'), left_on='nearest_pond_1_id', right_on='Pond ID_neighbor1', how='left')

# Merge with WQI data for the second nearest pond
merged_df = pd.merge(merged_df, wqi_df.add_suffix('_neighbor2'), left_on='nearest_pond_2_id', right_on='Pond ID_neighbor2', how='left')

# Merge with WQI data for the third nearest pond
merged_df = pd.merge(merged_df, wqi_df.add_suffix('_neighbor3'), left_on='nearest_pond_3_id', right_on='Pond ID_neighbor3', how='left')


print("Merged DataFrame Head:")
print(merged_df.head())

Merged DataFrame Head:
  kmz_file nearest_pond_1_id  nearest_pond_1_distance_m nearest_pond_2_id  \
0    1.kmz           1_106_2                     144.81            1_94_1   
1   10.kmz            1_93_2                     495.08            1_93_1   
2   11.kmz            1_93_1                     451.36           1_116_1   
3   12.kmz           1_101_2                     445.15            1_19_4   
4   13.kmz           1_129_3                     234.04           1_109_1   

   nearest_pond_2_distance_m nearest_pond_3_id  nearest_pond_3_distance_m  \
0                     177.44            1_98_1                     194.13   
1                     689.37            1_72_4                     879.41   
2                     505.04            1_93_2                     598.41   
3                     714.80            1_19_1                     725.69   
4                     242.00            1_19_1                     338.15   

   Pond ID Pond_Date Measurement Type  ...  \
0    

In [9]:
import numpy as np

# Set neighbor parameters to NaN if the distance is less than 10 meters
for i in range(1, 4):
    distance_col = f'nearest_pond_{i}_distance_m'
    for param in ['Dissolved_Oxygen', 'pH', 'Ammonia']:
        neighbor_param_col = f'{param}_neighbor{i}'
        # Check if the column exists before trying to modify it
        if neighbor_param_col in merged_df.columns:
            merged_df.loc[merged_df[distance_col] <= 10, neighbor_param_col] = np.nan

# We also need to filter the merged dataframe to only consider neighbor data that is on the same date
for i in range(1, 4):
    date_col_neighbor = f'Date of Data Collection_neighbor{i}'
    for param in ['Dissolved_Oxygen', 'pH', 'Ammonia']:
        neighbor_param_col = f'{param}_neighbor{i}'
        if date_col_neighbor in merged_df.columns and neighbor_param_col in merged_df.columns:
            merged_df.loc[merged_df['Pond_Date'].dt.date != merged_df[date_col_neighbor].dt.date, neighbor_param_col] = np.nan

print("Data after handling same pond condition and filtering by date:")
print(merged_df.head())

Data after handling same pond condition and filtering by date:
  kmz_file nearest_pond_1_id  nearest_pond_1_distance_m nearest_pond_2_id  \
0    1.kmz           1_106_2                     144.81            1_94_1   
1   10.kmz            1_93_2                     495.08            1_93_1   
2   11.kmz            1_93_1                     451.36           1_116_1   
3   12.kmz           1_101_2                     445.15            1_19_4   
4   13.kmz           1_129_3                     234.04           1_109_1   

   nearest_pond_2_distance_m nearest_pond_3_id  nearest_pond_3_distance_m  \
0                     177.44            1_98_1                     194.13   
1                     689.37            1_72_4                     879.41   
2                     505.04            1_93_2                     598.41   
3                     714.80            1_19_1                     725.69   
4                     242.00            1_19_1                     338.15   

   Pond ID 

In [10]:
# We need to extract the month from the main pond's date
merged_df['Month'] = merged_df['Pond_Date'].dt.month

# Define the parameters we want to analyze
params = ['Dissolved_Oxygen', 'pH', 'Ammonia']

# Create a dictionary to store the correlation results
correlation_results = {}

# Group by Pond ID and then by Month
for pond_id, group_pond in merged_df.groupby('Pond ID'):
    correlation_results[pond_id] = {}
    for month, group_month in group_pond.groupby('Month'):
        correlations = {}
        for param in params:
            pond_param = f'Pond_{param}'
            neighbor_params = [f'{param}_neighbor{i}' for i in range(1, 4) if f'{param}_neighbor{i}' in group_month.columns]
            
            # Combine the main pond's data with its neighbors' data for correlation calculation
            correlation_df = group_month[[pond_param] + neighbor_params].dropna()
            
            if not correlation_df.empty and len(correlation_df) > 1:
                correlations[param] = correlation_df.corr().iloc[0, 1:]
        
        if correlations:
            correlation_results[pond_id][month] = correlations

# Now, let's print the correlations for a specific pond and month as an example.
# You can change the pond_id and month to explore the results.
example_pond_id = 1
example_month = 1 # January

if example_pond_id in correlation_results and example_month in correlation_results[example_pond_id]:
    print(f"Correlations for Pond ID {example_pond_id} in Month {example_month}:")
    for param, corr in correlation_results[example_pond_id][example_month].items():
        print(f"\nParameter: {param}")
        print(corr)
else:
    print(f"No correlation data available for Pond ID {example_pond_id} in Month {example_month}.")

# To get a more general idea, let's calculate the average correlation across all ponds for each month
average_monthly_correlation = {}
for month in range(1, 13):
    monthly_corrs = {param: [] for param in params}
    for pond_id in correlation_results:
        if month in correlation_results[pond_id]:
            for param in params:
                if param in correlation_results[pond_id][month]:
                    monthly_corrs[param].append(correlation_results[pond_id][month][param].mean())
    
    average_monthly_correlation[month] = {param: np.nanmean(corrs) if corrs else np.nan for param, corrs in monthly_corrs.items()}

print("\nAverage Monthly Correlation Across All Ponds:")
avg_corr_df = pd.DataFrame(average_monthly_correlation).T
avg_corr_df.index.name = 'Month'
print(avg_corr_df)

No correlation data available for Pond ID 1 in Month 1.

Average Monthly Correlation Across All Ponds:
       Dissolved_Oxygen  pH  Ammonia
Month                               
1                   NaN NaN      NaN
2                   NaN NaN      NaN
3                   NaN NaN      NaN
4                   NaN NaN      NaN
5                   NaN NaN      NaN
6                   NaN NaN      NaN
7                   NaN NaN      NaN
8                   NaN NaN      NaN
9                   NaN NaN      NaN
10                  NaN NaN      NaN
11                  NaN NaN      NaN
12                  NaN NaN      NaN
